In [ ]:
# 1. Instalar bibliotecas
!pip install --upgrade google-cloud-bigquery pandas-gbq pyarrow

# 2. Autenticar no Google Cloud
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import re

In [ ]:


# Ler dataset (já precisa estar no /content/ depois do upload)
df = pd.read_csv("/content/netflix_titles.csv")

# Função simples para padronizar nomes de colunas
def snake_case(text):
    text = text.strip().lower()
    text = re.sub(r"[^\w\s]", "", text)
    return re.sub(r"\s+", "_", text)

df.columns = [snake_case(col) for col in df.columns]

# Converter coluna de datas
df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")

# Separar duration em valor e unidade
def split_duration(x):
    if pd.isna(x):
        return None, None
    match = re.match(r"^(\d+)\s*([A-Za-z]+)", str(x))
    if match:
        return int(match.group(1)), match.group(2).lower()
    return None, None

df[["duration_value", "duration_unit"]] = df["duration"].apply(lambda x: pd.Series(split_duration(x)))

# Ajuste de valores nulos e textos
df["country"] = df["country"].fillna("unknown")
df["rating"] = df["rating"].fillna("not_rated")

for col in ["type", "title", "director", "cast", "country", "listed_in", "description", "rating"]:
    df[col] = df[col].astype(str).str.strip().str.lower().replace("nan", None)

# Tabela base (sem duplicados)
titles_clean = df[
    ["show_id","type","title","director","cast","country","date_added",
     "release_year","rating","duration","duration_value","duration_unit",
     "listed_in","description"]
].drop_duplicates()

# Explode países
countries = []
for _, row in titles_clean.iterrows():
    for c in str(row["country"]).split(","):
        if c.strip():
            countries.append({"show_id": row["show_id"], "country": c.strip()})
titles_by_country = pd.DataFrame(countries)

# Explode gêneros
genres = []
for _, row in titles_clean.iterrows():
    for g in str(row["listed_in"]).split(","):
        if g.strip():
            genres.append({"show_id": row["show_id"], "genre": g.strip()})
titles_by_genre = pd.DataFrame(genres)

# Explode elenco
cast_rows = []
for _, row in titles_clean.iterrows():
    for a in str(row["cast"]).split(","):
        if a.strip():
            cast_rows.append({"show_id": row["show_id"], "actor": a.strip()})
titles_by_cast = pd.DataFrame(cast_rows)

# Exportar resultados
titles_clean.to_csv("/content/titles_clean.csv", index=False)
titles_by_country.to_csv("/content/titles_by_country.csv", index=False)
titles_by_genre.to_csv("/content/titles_by_genre.csv", index=False)
titles_by_cast.to_csv("/content/titles_by_cast.csv", index=False)


print("Arquivos tratados salvos com sucesso!")


Arquivos tratados salvos com sucesso!


In [ ]:
from pandas_gbq import to_gbq

PROJECT_ID = "analise-netflix"

# Tabelas para BigQuery
to_gbq(titles_clean, "netflix.titles_clean", project_id=PROJECT_ID, if_exists="replace")
to_gbq(titles_by_country, "netflix.titles_by_country", project_id=PROJECT_ID, if_exists="replace")
to_gbq(titles_by_genre, "netflix.titles_by_genre", project_id=PROJECT_ID, if_exists="replace")
to_gbq(titles_by_cast, "netflix.titles_by_cast", project_id=PROJECT_ID, if_exists="replace")

print("Tabelas enviadas para o BigQuery com sucesso!")


100%|██████████| 1/1 [00:00<00:00, 7681.88it/s]

Tabelas enviadas para o BigQuery com sucesso!
